In [ ]:
## Get 50.000 mags from AUs study, on zenodo
mamba activate zenodo_get

zenodo_get -h

https://zenodo.org/api/records/17252731
https://zenodo.org/records/17250845

In [ ]:
# unzip 
tar -xvzf your_file.tar.gz -C /path/to/destination/folder

for f in *mags.tar.gz 
do
tar -xvzf $f -C ../fasta
done

In [ ]:
# in: /home/ctbrown/scratch3/2025-other-pig-bins/annie-dl/
# make manysketch file

# sketch them
echo name,genome_filename,protein_filename > 251007_manysketch_Gaio2021.csv
for i in */*.fa
do
  echo "$i,$(realpath "$i"),"
done >> /home/ctbrown/scratch3/2025-other-pig-bins/annie-dl/fasta/251007_manysketch_Gaio2021.csv


# then for sketching
srun --account=ctbrowngrp -p med2 -J sketch -t 6:00:00 -c 60 --mem=80gb --pty bash

# sketch
mamba activate branchwater-skipmer
sourmash scripts manysketch -p dna,k=21,k=31,k=51,scaled=1000,abund \
-c 60 251007_manysketch_Gaio2021.csv -o MAGs_Gaio2021.all.zip


# then index them for a rocksdb 
sourmash index MAGs_Gaio2021.all.rocksdb -F rocksdb -k 31 MAGs_Gaio2021.all.zip

# then also make a pangenome db bc its faster 
mamba activate pangenomics_dev
sourmash scripts pangenome_merge \
MAGs_Gaio2021.all.zip -k 31 --scaled 1000 -o MAGs_Gaio2021.merged.k31.zip

In [ ]:
# tax for uncassified
# /home/ctbrown/scratch3/sourmash-midgie-raker/outputs.dg/rename/raw-sketch.unclassified.csv
# bins still in: /home/ctbrown/scratch3/2025-other-pig-bins/annie-dl/

srun --account=ctbrowngrp -p med2 -J gtdbtk -t 62:00:00 -c 52 --mem=120gb --pty bash

# do it in folders so that we dont do 300 at once bc the compyter doesnt like
mamba activate gtdbtk
gtdbtk classify_wf  --pplacer_cpus 16 \
--cpus 52 --batchfile gtdb_tk_btchfile.tsv \
--extension fasta \
--skip_ani_screen --out_dir taxonomy_MAGs_Gaio2021
